In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.util import random_noise
from skimage import filters
from skimage.morphology import disk
import warnings
warnings.simplefilter(action='ignore', category=DeprecationWarning)

def snpamount(x):
  return (x/90)+0.3

def draw_green(prev,frame):
    color = (0,255,0)
    for i in prev:
        x,y=i.ravel() #corner coordinates
        img = cv2.circle(frame,(x,y),3,color,-1) #draw a green circle of radius=3 centered at (x,y)
    plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

def find_corners(current,first_frame_corners,new_corners):
    array = np.asarray(new_corners) #convert list to np.array
    for i in current: #for all corners in current frame
        if np.all(first_frame_corners!=i) and np.all(array != i): #if corner in question is not in the first frame or already checked
            new_corners.append(i) #then append in list

#1st Question
def resize_frame(frame):
    dimensions = frame.shape
    new_frame = cv2.resize(frame, (int(dimensions[1]/2),int(dimensions[0]/2)))
    return new_frame

#4th Question
def first_frame(cap,feature_params,lk_params,detector_type):
    
    ret, frame = cap.read()
    resized_frame = resize_frame(frame)
    gray_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)
    if (detector_type==1):
        first_frame_corners = cv2.goodFeaturesToTrack(gray_frame, mask = None, **feature_params, useHarrisDetector=True)
    else:
        first_frame_corners = cv2.goodFeaturesToTrack(gray_frame, mask = None, **feature_params)
    mask = np.zeros_like(resized_frame)
    
    while(cap.isOpened()):
        ret, frame = cap.read()
        if frame is None: #if there are no more frames break
            break
        resized_frame = resize_frame(frame)
        gray = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)
        new, status, error = cv2.calcOpticalFlowPyrLK(gray_frame, gray, first_frame_corners, None, **lk_params)
        good_old = first_frame_corners[status == 1]
        good_new = new[status == 1] #note corners whose optical flow has been found
        color = (0,255,0)
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()
            c, d = old.ravel() #corner coordinates in current and previous frame
            mask = cv2.line(mask, (a, b), (c, d), color, 2) #draw green line between the two positions
            resized_frame = cv2.circle(resized_frame, (a, b), 3, color, -1) #note corner 
        output = cv2.add(resized_frame, mask)
        gray_frame = gray.copy() #current frame is now the old frame
        first_frame_corners = good_new.reshape(-1, 1, 2)
        cv2.imshow("sparse optical flow", output)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

#5th Question
def next_frames(cap):
    ret, frame = cap.read()
    resized_frame = resize_frame(frame)
    gray_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)
    feature_params = dict(maxCorners = 300, qualityLevel = 0.1, minDistance = 4, blockSize = 7)
    lk_params = dict(winSize = (30,30), maxLevel = 4, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
    mask = np.zeros_like(resized_frame)
    new_corners = []
    counted_frames = 0
    first_frame_corners = cv2.goodFeaturesToTrack(gray_frame, mask = None, **feature_params)
    
    while(cap.isOpened()):
        ret, frame = cap.read()
        if frame is None: #if there are no more frames break
            break
        resized_frame = resize_frame(frame)
        gray = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)
        if ((counted_frames==0) or (counted_frames%3==0)): #check for feature points every 3 frames
            current = cv2.goodFeaturesToTrack(gray, mask = None, **feature_params) #feature points of current frame
            find_corners(current, first_frame_corners, new_corners) #add them to the existing points
            new_corners_array= np.asarray(new_corners) #convert list of points to np.array
        new, status, error = cv2.calcOpticalFlowPyrLK(gray_frame, gray, new_corners_array, None, **lk_params)
        good_old = new_corners_array[status == 1]
        good_new = new[status == 1] #note corners whose optical flow has been found
        color = (0,255,0)
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()
            c, d = old.ravel() #corner coordinates in current and previous frame
            if(abs(a-c)>1 or abs(b-d)>1): #check for movement in x or y direction (transition larger than 1 pixel)
                mask = cv2.line(mask, (a, b), (c, d), color, 2) #corner coordinates in current and previous frame
                resized_frame = cv2.circle(resized_frame, (a, b), 3, color, -1) #draw green line between the two positions
        output = cv2.add(resized_frame, mask)
        gray_frame = gray.copy() #current frame is now the old frame
        new_corners_array=good_new.reshape(-1,1,2)
        cv2.imshow("sparse optical flow", output)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        counted_frames = counted_frames+1 
    cap.release()
    cv2.destroyAllWindows()

#6th Question  
def next_frames_noise(cap):
    ret, frame = cap.read()
    resized_frame = resize_frame(frame)
    gray_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)
    snp_initial = random_noise(gray_frame,mode='s&p',amount=snpamount(1)) #add snp noise
    snp_initial = np.array(255*snp_initial,dtype='uint8') #convert to uint8
    feature_params = dict(maxCorners = 300, qualityLevel = 0.1, minDistance = 4, blockSize = 7)
    lk_params = dict(winSize = (30,30), maxLevel = 4, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
    mask = np.zeros_like(resized_frame)
    new_corners = []
    counted_frames = 0
    first_frame_corners = cv2.goodFeaturesToTrack(snp_initial, mask = None, **feature_params)
    while(cap.isOpened()):
        ret, frame = cap.read()
        if frame is None:
            break
        resized_frame = resize_frame(frame)
        gray = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)
        snp_gray = random_noise(gray,mode='s&p',amount=snpamount(1)) #add snp noise 
        snp_gray = np.array(255*snp_gray,dtype='uint8') #convert to uint8
        if ((counted_frames==0) or (counted_frames%3==0)): #check for feature points every 3 frames
            current = cv2.goodFeaturesToTrack(snp_gray, mask = None, **feature_params) #feature points of current frame
            find_corners(current, first_frame_corners, new_corners) #add them to the existing points
            new_corners_array= np.asarray(new_corners) #convert list of points to np.array
        new, status, error = cv2.calcOpticalFlowPyrLK(snp_initial, snp_gray, new_corners_array, None, **lk_params)
        good_old = new_corners_array[status == 1]
        good_new = new[status == 1] #note corners whose optical flow has been found
        color = (0,255,0)
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()
            c, d = old.ravel() #corner coordinates in current and previous frame
            if(abs(a-c)>1 or abs(b-d)>1): #check for movement in x or y direction (transition larger than 1 pixel)
                mask = cv2.line(mask, (a, b), (c, d), color, 2) #corner coordinates in current and previous frame
                resized_frame = cv2.circle(resized_frame, (a, b), 3, color, -1) #draw green line between the two positions
        output = cv2.add(resized_frame, mask)
        snp_initial = snp_gray.copy() #current frame is now the old frame
        new_corners_array=good_new.reshape(-1,1,2)
        cv2.imshow("sparse optical flow", output)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        counted_frames = counted_frames+1 
    cap.release()
    cv2.destroyAllWindows()

    
#7th Question    
def next_frames_denoised(cap):
    ret, frame = cap.read()
    resized_frame = resize_frame(frame)
    gray_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)
    snp_initial = random_noise(gray_frame,mode='s&p',amount=snpamount(1)) #add snp noise
    snp_initial = np.array(255*snp_initial,dtype='uint8') #convert to uint8
    denoised_initial = filters.rank.median(snp_initial, disk(3)) #apply median filter 
    feature_params = dict(maxCorners = 300, qualityLevel = 0.1, minDistance = 4, blockSize = 7)
    lk_params = dict(winSize = (30,30), maxLevel = 4, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
    mask = np.zeros_like(resized_frame)
    new_corners = []
    counted_frames = 0
    first_frame_corners = cv2.goodFeaturesToTrack(denoised_initial, mask = None, **feature_params)
    while(cap.isOpened()):
        ret, frame = cap.read()
        if frame is None:
            break
        resized_frame = resize_frame(frame)
        gray = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)
        snp_gray = random_noise(gray,mode='s&p',amount=snpamount(1)) #add snp noise
        snp_gray = np.array(255*snp_gray,dtype='uint8') #convert to uint8
        denoised_gray = filters.rank.median(snp_gray, disk(3)) #apply median filter
        if ((counted_frames==0) or (counted_frames%3==0)):  #check for feature points every 3 frames
            current = cv2.goodFeaturesToTrack(denoised_gray, mask = None, **feature_params) #feature points of current frame
            find_corners(current, first_frame_corners, new_corners) #add them to the existing points
            new_corners_array= np.asarray(new_corners) #convert list of points to np.array
        new, status, error = cv2.calcOpticalFlowPyrLK(denoised_initial, denoised_gray, new_corners_array, None, **lk_params)
        good_old = new_corners_array[status == 1]
        good_new = new[status == 1] #note corners whose optical flow has been found
        color = (0,255,0)
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()
            c, d = old.ravel() #corner coordinates in current and previous frame
            if(abs(a-c)>1 or abs(b-d)>1): #check for movement in x or y direction (transition larger than 1 pixel)
                mask = cv2.line(mask, (a, b), (c, d), color, 2) #corner coordinates in current and previous frame
                resized_frame = cv2.circle(resized_frame, (a, b), 3, color, -1) #draw green line between the two positions
        output = cv2.add(resized_frame, mask)
        denoised_initial = denoised_gray.copy() #current frame is now the old frame
        new_corners_array=good_new.reshape(-1,1,2)
        cv2.imshow("sparse optical flow", output)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        counted_frames = counted_frames+1 
    cap.release()
    cv2.destroyAllWindows()
    
cap = cv2.VideoCapture("video.mp4")
ret, frame = cap.read()

#2nd Question
resized_frame = resize_frame(frame)
gray_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)

#3rd Question
#Harris Corner Detector
#1st Trial
feature_params = dict(maxCorners = 300, qualityLevel = 0.1, minDistance = 4, blockSize = 7)
prev = cv2.goodFeaturesToTrack(gray_frame, mask = None, **feature_params, useHarrisDetector = True)
draw_green(prev, resized_frame)
#2nd Trial
feature_params = dict(maxCorners = 300, qualityLevel = 0.2, minDistance = 2, blockSize = 7)
prev = cv2.goodFeaturesToTrack(gray_frame, mask = None, **feature_params, useHarrisDetector = True)
resized_frame = resize_frame(frame)
draw_green(prev, resized_frame)
#Shi-Tomasi Detector
#1st Trial-choice
feature_params = dict(maxCorners = 300, qualityLevel = 0.1, minDistance = 4, blockSize = 7)
prev = cv2.goodFeaturesToTrack(gray_frame, mask = None, **feature_params)
resized_frame = resize_frame(frame)
draw_green(prev, resized_frame)
#2nd Trial
feature_params = dict(maxCorners = 300, qualityLevel = 0.2, minDistance = 2, blockSize = 7)
prev = cv2.goodFeaturesToTrack(gray_frame, mask = None, **feature_params)
resized_frame = resize_frame(frame)
draw_green(prev, resized_frame)

cap.release()

#4th Question
#LK PARAMS ((15,15),2,(30,0.01))
lk_params = dict(winSize = (15,15), maxLevel = 2, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01))
#1st Trial - Harris Corner Detector
feature_params = dict(maxCorners = 300, qualityLevel = 0.1, minDistance = 4, blockSize = 7)
cap = cv2.VideoCapture("video.mp4")
first_frame(cap,feature_params,lk_params,1)
#2nd Trial - Harris Corner Detector
feature_params = dict(maxCorners = 300, qualityLevel = 0.2, minDistance = 2, blockSize = 7)
cap = cv2.VideoCapture("video.mp4")
first_frame(cap,feature_params,lk_params,1)
#1st Trial - ShiTomasi Detector - CHOICE
feature_params = dict(maxCorners = 300, qualityLevel = 0.1, minDistance = 4, blockSize = 7)
cap = cv2.VideoCapture("video.mp4")
first_frame(cap,feature_params,lk_params,0)
#2nd Trial - ShiTomasi Detector
feature_params = dict(maxCorners = 300, qualityLevel = 0.2, minDistance = 2, blockSize = 7)
cap = cv2.VideoCapture("video.mp4")
first_frame(cap,feature_params,lk_params,0)

#LK PARAMS ((30,30),4,(10,0.03))
lk_params = dict(winSize = (30,30), maxLevel = 4, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
#1st Trial - Harris Corner Detector
feature_params = dict(maxCorners = 300, qualityLevel = 0.1, minDistance = 4, blockSize = 7)
cap = cv2.VideoCapture("video.mp4")
first_frame(cap,feature_params,lk_params,1)
#2nd Trial - Harris Corner Detector
feature_params = dict(maxCorners = 300, qualityLevel = 0.2, minDistance = 2, blockSize = 7)
cap = cv2.VideoCapture("video.mp4")
first_frame(cap,feature_params,lk_params,1)
#1st Trial - ShiTomasi Detector - CHOICE
feature_params = dict(maxCorners = 300, qualityLevel = 0.1, minDistance = 4, blockSize = 7)
cap = cv2.VideoCapture("video.mp4")
first_frame(cap,feature_params,lk_params,0)
#2nd Trial - ShiTomasi Detector
feature_params = dict(maxCorners = 300, qualityLevel = 0.2, minDistance = 2, blockSize = 7)
cap = cv2.VideoCapture("video.mp4")
first_frame(cap,feature_params,lk_params,0)

#5th Question
cap.release()
cap = cv2.VideoCapture("video.mp4")
next_frames(cap)

#6th Question
cap.release()
cap = cv2.VideoCapture("video.mp4")
next_frames_noise(cap)

#7th Question
cap.release()
cap = cv2.VideoCapture("video.mp4")
next_frames_denoised(cap)